In [1]:
import pandas as pd
import tensorflow as tf
import numpy
df_train_labels_original = pd.read_csv('train_labels.csv',low_memory=False, dtype= {
    'damage_grade':'uint8'
}).set_index('building_id')
df_train_values_original = pd.read_csv('train_values.csv',low_memory=False, dtype= {
    'geo_level_1_id':'category', 
    'geo_level_2_id':'category',
    'geo_level_3_id':'category', 
    'count_floors_pre_eq':'uint8',
    'age':'uint16',
    'area_percentage':'uint16', 
    'height_percentage':'uint16', 
    'land_surface_condition':'category', 
    'foundation_type':'category',
    'roof_type':'category',
    'ground_floor_type':'category',
    'other_floor_type':'category',
    'position':'category',
    'plan_configuration':'category', 
    'has_superstructure_adobe_mud':'uint8',
    'has_superstructure_mud_mortar_stone':'uint8',
    'has_superstructure_stone_flag':'uint8',
    'has_superstructure_cement_mortar_stone':'uint8', 
    'has_superstructure_mud_mortar_brick':'uint8', 
    'has_superstructure_cement_mortar_brick':'uint8', 
    'has_superstructure_timber':'uint8', 
    'has_superstructure_bamboo':'uint8',
    'has_superstructure_rc_non_engineered':'uint8',
    'has_superstructure_rc_engineered':'uint8',
    'has_superstructure_other':'uint8', 
    'legal_ownership_status':'category',
    'count_families':'uint16', 
    'has_secondary_use':'uint8', 
    'has_secondary_use_agriculture':'uint8', 
    'has_secondary_use_hotel':'uint8',
    'has_secondary_use_rental':'uint8',
    'has_secondary_use_institution':'uint8',
    'has_secondary_use_school':'uint8', 
    'has_secondary_use_industry':'uint8', 
    'has_secondary_use_health_post':'uint8', 
    'has_secondary_use_gov_office':'uint8', 
    'has_secondary_use_use_police':'uint8', 
    'has_secondary_use_other':'uint8',
}).set_index('building_id')


pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
df = df_train_values_original.join(df_train_labels_original,how="inner")
df

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688636,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,2
669485,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
602512,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,3


In [3]:
def category_encoding(dataframe, column_name):
    new_column_names = {
        1: column_name+'_1',
        2: column_name+'_2',
    }
    cross = pd.crosstab(dataframe[column_name], dataframe['damage_grade']).drop(columns=[3]).rename(columns=new_column_names)
    prob = cross.divide(cross.apply('sum',axis=1),axis=0).reset_index()
    return dataframe.reset_index().merge(prob,on=column_name).set_index('building_id').drop(columns=[column_name])

In [4]:
df_geo = category_encoding(df.drop(columns=['geo_level_1_id', 'geo_level_3_id']), 'geo_level_2_id')
df_lsc = category_encoding(df_geo, 'land_surface_condition')
df_ft  = category_encoding(df_lsc, 'foundation_type')
df_rt  = category_encoding(df_ft , 'roof_type')
df_gft = category_encoding(df_rt , 'ground_floor_type')
df_oft = category_encoding(df_gft, 'other_floor_type')
df_p   = category_encoding(df_oft, 'position')
df_pc  = category_encoding(df_p  , 'plan_configuration')
df_los = category_encoding(df_pc , 'legal_ownership_status')
df_los

,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,...,ground_floor_type_1,ground_floor_type_2,other_floor_type_1,other_floor_type_2,position_1,position_2,plan_configuration_1,plan_configuration_2,legal_ownership_status_1,legal_ownership_status_2
building_id,,,,,,,,,,,,,,,,,,,,,
802906,2,30,6,5,1,1,0,0,0,0,...,0.09,0.91,0.07,0.93,0.13,0.87,0.14,0.86,0.14,0.86
1003118,2,15,6,6,0,1,0,0,0,0,...,0.09,0.91,0.07,0.93,0.13,0.87,0.14,0.86,0.14,0.86
496583,2,5,8,6,0,0,0,0,0,1,...,0.09,0.91,0.07,0.93,0.13,0.87,0.14,0.86,0.14,0.86
32498,2,15,5,6,0,1,0,0,0,0,...,0.09,0.91,0.07,0.93,0.13,0.87,0.14,0.86,0.14,0.86
683368,2,0,7,5,0,1,0,0,0,0,...,0.09,0.91,0.07,0.93,0.13,0.87,0.14,0.86,0.14,0.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661864,2,25,9,5,0,1,0,0,0,0,...,0.09,0.91,0.13,0.87,0.15,0.85,0.19,0.81,0.09,0.91
126822,3,30,14,6,0,1,0,0,0,0,...,0.09,0.91,0.07,0.93,0.17,0.83,0.00,1.00,0.09,0.91
645560,2,40,5,7,0,1,0,0,0,0,...,0.09,0.91,0.07,0.93,0.13,0.87,0.29,0.71,0.09,0.91


In [5]:
target = df.pop('damage_grade').apply(lambda x: x-1)
target

building_id
802906    2
28830     1
94947     2
590882    1
201944    2
         ..
688636    1
669485    2
602512    2
151409    1
747594    2
Name: damage_grade, Length: 260601, dtype: int64

In [6]:
numpy_database = df_los.to_numpy()
numpy_database

array([[ 2.        , 30.        ,  6.        , ...,  0.8594369 ,
         0.13982358,  0.86017642],
       [ 2.        , 15.        ,  6.        , ...,  0.8594369 ,
         0.13982358,  0.86017642],
       [ 2.        ,  5.        ,  8.        , ...,  0.8594369 ,
         0.13982358,  0.86017642],
       ...,
       [ 2.        , 40.        ,  5.        , ...,  0.70588235,
         0.09065551,  0.90934449],
       [ 3.        , 25.        , 10.        , ...,  0.70547945,
         0.09065551,  0.90934449],
       [ 3.        , 25.        ,  8.        , ...,  0.70547945,
         0.09065551,  0.90934449]])

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((df_los, target.values))
dataset

<TensorSliceDataset shapes: ((46,), ()), types: (tf.float64, tf.int64)>

In [8]:
train_dataset = dataset.shuffle(len(df)).batch(1)
train_dataset

<BatchDataset shapes: ((None, 46), (None,)), types: (tf.float64, tf.int64)>

In [9]:
with tf.device('/CPU:0'):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(92, activation='relu'),
        tf.keras.layers.Dense(units=3, activation='sigmoid')
      ])
predictions = model(numpy_database[:1]).numpy()
predictions

array([[0.27831948, 0.03336347, 0.9723332 ]], dtype=float32)

In [12]:
with tf.device('/CPU:0'):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    model.compile(optimizer=tf.keras.optimizers.SGD(),
                  loss=loss_fn,
                  metrics=['accuracy'])
    model.fit(train_dataset, epochs=1)

260601/260601 [==============================] - 342s 1ms/step - loss: nan - accuracy: 0.0964


In [13]:
with tf.device('/CPU:0'):
    model.fit(train_dataset, epochs=10)
model.save('modelos/NNModel3')

Epoch 1/10
260601/260601 [==============================] - 339s 1ms/step - loss: nan - accuracy: 0.0964
Epoch 2/10
260601/260601 [==============================] - 340s 1ms/step - loss: nan - accuracy: 0.0964
Epoch 3/10
260601/260601 [==============================] - 339s 1ms/step - loss: nan - accuracy: 0.0964
Epoch 4/10
260601/260601 [==============================] - 342s 1ms/step - loss: nan - accuracy: 0.0964
Epoch 5/10
260601/260601 [==============================] - 341s 1ms/step - loss: nan - accuracy: 0.0964
Epoch 6/10
260601/260601 [==============================] - 341s 1ms/step - loss: nan - accuracy: 0.0964
Epoch 7/10
260601/260601 [==============================] - 340s 1ms/step - loss: nan - accuracy: 0.0964
Epoch 8/10
260601/260601 [==============================] - 342s 1ms/step - loss: nan - accuracy: 0.0964
Epoch 9/10
260601/260601 [==============================] - 342s 1ms/step - loss: nan - accuracy: 0.0964
Epoch 10/10
260601/260601 [============================